In [ ]:
#To do
#Try disorder strength 1 which is defineltely ergodic phase and also try strengh something like 30

In [ ]:
import netket as nk
from numpy.lib.function_base import append
from itertools import permutations, combinations
from netket.vqs.mc import get_local_kernel_arguments, get_local_kernel
import time
import matplotlib.pyplot as plt
import json
import numpy as np
from numpy.linalg import eig
# from flax.core.frozen_dict import FrozenDict
import flax
from netket.optimizer.qgt import QGTJacobianPyTree


from typing import Callable, Tuple
from functools import partial

import jax
from jax import numpy as jnp

from netket.stats import statistics as mpi_statistics, mean as mpi_mean, Stats
from netket.utils.types import PyTree
from netket.operator.spin import sigmax, sigmay,sigmaz

import matplotlib.pyplot as plt
from tqdm import tqdm
################################################################
################################################################
################################################################
###############ED는 sparse matrix가지고 한다 !!!###################
from scipy.sparse.linalg import eigsh
################################################################
################################################################
################################################################
import os
import pandas as pd


from alpha_mcmc import alpha_mcmc
from calculate_variance import calculate_variance


In [ ]:
import os
os.environ["JAX_PLATFORM_NAME"] = "cpu"

In [ ]:
# Define a 1d chain
L = 10
g = nk.graph.Hypercube(length=L, n_dim=1, pbc=True)
#g.translation_group
#g.point_group
#g.rotation_group
#g.space_group

hi = nk.hilbert.Spin(s=0.5, total_sz=0, N=g.n_nodes)
# hi = nk.hilbert.Spin(s=0.5, N=g.n_nodes)

# calling the Heisenberg Hamiltonian
ha = nk.operator.Heisenberg(hilbert=hi, graph=g)
b=1
a=-1
Gamma = (b-a) * np.random.random_sample(L) + a

################################################################################
################################################################################
################################################################################
#이렇게 정의하는게 더 빨랑
ha = sum([ sigmax(hi,i)*sigmax(hi,(i+1)%L)
         + sigmay(hi,i)*sigmay(hi,(i+1)%L)
         + sigmaz(hi,i)*sigmaz(hi,(i+1)%L)
           for i in range(L)])
################################################################################
################################################################################
################################################################################
ha = ha + sum([Gamma[i]*sigmaz(hi,i) for i in range(L)])

leng = np.shape(hi.all_states())[0]
print(leng)

In [ ]:
E, vec = eigsh(ha.to_sparse(), k = 200, which="LA")

In [ ]:
np.shape(vec)

In [ ]:
sma = max(E)-8
big = max(E)-7
Et = np.random.uniform(low=sma, high=big, size=None)
plt.scatter(np.arange(len(E)),np.sort(E),s=10)
plt.plot([0,len(E)],[Et,Et],'r')
plt.ylim([Et-0.1,Et+0.1])
print(Et)

In [ ]:
Target = (ha-Et)@(ha-Et)

In [ ]:
def Rule_Full(eta, n_steps_):
    ma = nk.models.RBM(alpha=2, param_dtype=complex)
    vs = nk.vqs.FullSumState(hi, ma)
    op = nk.optimizer.Adam(learning_rate=eta)
    sr = nk.optimizer.SR(diag_shift=0.1, holomorphic=True)

    gs = nk.VMC(
      hamiltonian=Target,
      optimizer=op,
      preconditioner=sr,
      variational_state=vs)

    start = time.time()
    gs.run(n_steps_, out='RBM')
    end = time.time()
    H_Et_squred = vs.expect((ha-Et)@(ha-Et)).mean.real
    Energy = vs.expect(ha).mean.real
    Variance = calculate_variance(ha, vs)
    result = [H_Et_squred, Energy, Variance]
    return result

def Rule_Exchange(eta, n_steps_, n_samples_,batch):
    ma = nk.models.RBM(alpha=2, param_dtype=complex)
#     sa = nk.sampler.MetropolisHamiltonian(hi, ha)
    sa = nk.sampler.MetropolisExchange(hi, graph=g, n_chains=batch)
    vs = nk.vqs.MCState(sa, ma, n_samples = n_samples_)

    op = nk.optimizer.Sgd(learning_rate=eta)
    sr = nk.optimizer.SR(diag_shift=0.1, holomorphic=True)

    gs = nk.VMC(
      hamiltonian=Target,
      optimizer=op,
      preconditioner=sr,
      variational_state=vs)

    start = time.time()
    gs.run(n_steps_, out='RBM')
    end = time.time()
    H_Et_squred = vs.expect((ha-Et)@(ha-Et)).mean.real
    Energy = vs.expect(ha).mean.real
    Variance = calculate_variance(ha, vs)
    result = [H_Et_squred, Energy, Variance]
    return result


In [ ]:
A10 = Rule_Full(eta=0.01, n_steps_=10000)

In [ ]:
def alpha_depen(n_steps, alpha_):
  learning_rate = 0.001
  ma = nk.models.RBM(alpha=alpha_, param_dtype=complex)
  vs = nk.vqs.FullSumState(hi, ma)
  op = nk.optimizer.Sgd(learning_rate=learning_rate)
  sr = nk.optimizer.SR(diag_shift=0.001, holomorphic=True)
    
#     ma = nk.models.RBM(alpha=2, param_dtype=complex)
#     vs = nk.vqs.FullSumState(hi, ma)
#     op = nk.optimizer.Adam(learning_rate=eta)
#     sr = nk.optimizer.SR(diag_shift=0.1, holomorphic=True)
     
  gs = nk.VMC(
      hamiltonian=Target,
      optimizer=op,
      preconditioner=sr,
      variational_state=vs)

  start = time.time()
  gs.run(n_steps, out='RBM')
  end = time.time()
  data=json.load(open("RBM.log"))
  energy = vs.expect(ha).mean.real
  variance = vs.expect(ha@ha).mean.real-vs.expect(ha).mean.real**2
  print([energy, variance])
  return energy, variance;

In [ ]:
e2 , v2 = alpha_depen(30000,2)

In [ ]:
v2

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


NameError: name 'hi' is not defined